In [ ]:
import requests
import base64

# Step 1: Set your client id, client secret, and redirect uri
CLIENT_ID = '6b7e2e58b62d4010a958a1f14c3a2319'
CLIENT_SECRET = 'fcffd00829104392be509b4531c95c5d'
REDIRECT_URI = 'http://localhost:8880/callback'

# Step 2: Get the access token
def get_access_token():
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    response = requests.post("https://accounts.spotify.com/api/token", data={
        'grant_type': 'client_credentials'
    }, headers={
        "Authorization": f"Basic {auth_header}"
    })
    return response.json()["access_token"]

# Step 3: Use the access token to make requests
def get_artist_info(artist_name):
    token = get_access_token()
    response = requests.get(f"https://api.spotify.com/v1/search?q={artist_name}&type=artist", headers={
        "Authorization": f"Bearer {token}"
    })
    return response.json()

# Example usage:
artist_info = get_artist_info("Adele")
print(artist_info)

The URL the user will go to give us info is below. This is just for beta and closed development.When the user gives us their info they will be met with localhost can't be connected to or something along the lines. For now the link is code= whatever in the url, you can copy and paste it. In the future when we get our own url, we will be able to have a function that just grabs the code without us interfering (callback function). The access token lasts for an hour and the auth code is one use. So if you used the auth code snippet once, please don't refresh it, it will give an error.

https://accounts.spotify.com/en/authorize?client_id=6b7e2e58b62d4010a958a1f14c3a2319&response_type=code&redirect_uri=http://localhost:8880/callback&scope=user-top-read%20user-read-private%20playlist-read-private

In [ ]:
import requests
import base64

def get_access_token_from_code(auth_code):
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    response = requests.post("https://accounts.spotify.com/api/token", data={
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': REDIRECT_URI
    }, headers={
        "Authorization": f"Basic {auth_header}"
    })
    return response.json().get("access_token"), response.json().get("refresh_token")

def refresh_access_token(refresh_token):
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    response = requests.post("https://accounts.spotify.com/api/token", data={
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }, headers={
        "Authorization": f"Basic {auth_header}"
    })
    return response.json().get("access_token")
CLIENT_ID = '6b7e2e58b62d4010a958a1f14c3a2319'
CLIENT_SECRET = 'fcffd00829104392be509b4531c95c5d'
REDIRECT_URI = 'http://localhost:8880/callback'
AUTHORIZATION_CODE = 'AQAnHASCGjkw3ovSTj_um4E7TBtdLyD2WNocIS9PbeQKanyjS6jipkKk7kHDFiRUPqUxmECd1jho0ESQlCpbsHeRhWke2xB1yvJoJ_9zsdHYCg8OlnNpbaO1WwVLip9WrW6o6c_zmSICOQNZvRj7q5VKIl3EBiE8_Oem1A3X22UwoFOkFIjL017O_hz5us_hfXN3PDmjydb2bZFA1AqGiryT0TUSq9FJ2TTGNOX3ENJWls8w-D5owW-hRQ'
access_token, refresh_token = get_access_token_from_code(AUTHORIZATION_CODE)

headers = {
    "Authorization": f"Bearer {access_token}"
}



In [ ]:
headers["Authorization"] = f"Bearer {access_token}"
response = requests.get("https://api.spotify.com/v1/me/top/tracks?time_range=medium_term&limit=30", headers=headers)

top_tracks = response.json()
for track in top_tracks['items']:
    print(track['name'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import pairwise_distances

# Step 1: Fetch Audio Features for US Top Hits
US_TOP_HITS_PLAYLIST_ID ="37i9dQZF1DXcBWIGoYBM5M?si=beaf719fd1f64d09" # replace with the actual ID
response = requests.get(f"https://api.spotify.com/v1/playlists/{US_TOP_HITS_PLAYLIST_ID}/tracks", headers=headers)
us_top_hits_tracks = response.json()

us_top_hits_track_ids = [track['track']['id'] for track in us_top_hits_tracks['tracks']['items']]

features_response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={','.join(us_top_hits_track_ids)}", headers=headers)
us_top_hits_features = features_response.json()

# Step 2: Data Preparation
my_tracks_df = pd.DataFrame(features['audio_features'])  # Assuming 'features' is defined somewhere in your original code
us_top_hits_df = pd.DataFrame(us_top_hits_features['audio_features'])

selected_features = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X_my_tracks = my_tracks_df[selected_features]
X_us_top_hits = us_top_hits_df[selected_features]

scaler = StandardScaler().fit(X_my_tracks)
X_my_tracks = scaler.transform(X_my_tracks)
X_us_top_hits = scaler.transform(X_us_top_hits)

# Compute Pairwise Distances
pairwise_dists = pairwise_distances(X_us_top_hits)
unique_distances = pairwise_dists[np.triu_indices(pairwise_dists.shape[0], k=1)]
threshold = np.percentile(unique_distances, 25)

# Step 3: k-Nearest Neighbors (k-NN)
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_us_top_hits)

distances, indices = knn.kneighbors(X_my_tracks)

# Step 4: Recommend Songs
sorted_recommendations = sorted(zip(distances.flatten(), indices.flatten()), key=lambda x: x[0])
filtered_recommendations = [(dist, idx) for dist, idx in sorted_recommendations if dist <= threshold]
top_10_indices = [idx for _, idx in filtered_recommendations[:10]]
top_10_songs = [f"{us_top_hits_tracks['tracks']['items'][i]['track']['name']} by {us_top_hits_tracks['tracks']['items'][i]['track']['artists'][0]['name']}" for i in top_10_indices]
print(top_10_songs)




In [ ]:

# Compute Pairwise Distances with Cosine Similarity
pairwise_dists = pairwise_distances(X_us_top_hits, metric='cosine')
unique_distances = pairwise_dists[np.triu_indices(pairwise_dists.shape[0], k=1)]
threshold = np.percentile(unique_distances, 25)  # Adjust this if necessary

# Step 3: k-Nearest Neighbors (k-NN) with Cosine Similarity
knn = NearestNeighbors(n_neighbors=5, metric='cosine')
knn.fit(X_us_top_hits)

distances, indices = knn.kneighbors(X_my_tracks)

# Step 4: Recommend Songs
sorted_recommendations = sorted(zip(distances.flatten(), indices.flatten()), key=lambda x: x[0])
filtered_recommendations = [(dist, idx) for dist, idx in sorted_recommendations if dist <= threshold]
top_10_indices = [idx for _, idx in filtered_recommendations[:10]]
top_10_songs = [f"{us_top_hits_tracks['tracks']['items'][i]['track']['name']} by {us_top_hits_tracks['tracks']['items'][i]['track']['artists'][0]['name']}" for i in top_10_indices]
print(top_10_songs)


In [18]:
from flask import Flask, render_template, redirect, session, make_response, url_for, request
from urllib.parse import urlencode
import os
import requests
import logging
# flask stuff 
app = Flask(__name__)
app.secret_key = os.urandom(24)  # Secret key for Flask session

# Assuming you have a config.py with CLIENT_ID, REDIRECT_URI, SCOPE, and AUTHORIZATION
app.config.from_pyfile('config.py')


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/authorize')
def authorize():
    client_id = app.config['CLIENT_ID']
    redirect_uri = app.config['REDIRECT_URI']
    scope = app.config['SCOPE']
    state_key = os.urandom(15).hex()  # Creating a random state key
    session['state_key'] = state_key

    authorize_url = 'https://accounts.spotify.com/en/authorize?'
    params = {'response_type': 'code', 'client_id': client_id,
              'redirect_uri': redirect_uri, 'scope': scope,
              'state': state_key}
    query_params = urlencode(params)
    response = make_response(redirect(authorize_url + query_params))
    return response


def getToken(code):
    token_url = 'https://accounts.spotify.com/api/token'
    authorization = app.config['AUTHORIZATION']
    redirect_uri = app.config['REDIRECT_URI']
    headers = {'Authorization': f"Basic {authorization}",  # Updated the Authorization header
               'Accept': 'application/json',
               'Content-Type': 'application/x-www-form-urlencoded'}
    body = {'code': code, 'redirect_uri': redirect_uri,
            'grant_type': 'authorization_code'}
    post_response = requests.post(token_url, headers=headers, data=body)
    if post_response.status_code == 200:
        pr = post_response.json()
        return pr['access_token'], pr['refresh_token'], pr['expires_in']
    else:
        logging.error('getToken:' + str(post_response.status_code))
        return None


def get_top_tracks(access_token, limit=50):
    headers = {
        'Authorization': f"Bearer {access_token}"
    }
    response = requests.get(f"https://api.spotify.com/v1/me/top/tracks?limit={limit}", headers=headers)

    if response.status_code == 200:
        return response.json()['items']
    else:
        logging.error(f"get_top_tracks: {response.status_code}")
        return []


@app.route('/callback/')
def callback():
    # Check for error
    error = request.args.get('error')
    if error:
        # Handle the error based on your requirements
        logging.error(f"Error during authorization: {error}")
        return render_template('error.html', error=error)

    # Check state to prevent CSRF
    state = request.args.get('state')
    if not state or state != session.get('state_key'):
        logging.error("State mismatch error")
        return render_template('error.html', error="State mismatch error")

    # Get the code from the callback URL
    code = request.args.get('code')

    # Use the code to get the access token
    access_token, refresh_token, expires_in = getToken(code)

    # Store the access token, refresh token, and expiration in the session or database
    session['access_token'] = access_token
    session['refresh_token'] = refresh_token
    session['expires_in'] = expires_in

    # Get the top tracks
    top_tracks = get_top_tracks(access_token)

    # Render a template displaying the top tracks
    return render_template('top_tracks.html', tracks=top_tracks)


if __name__ == "__main__":
    app.run(debug=False)

    
 



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [03/Oct/2023 21:52:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Oct/2023 21:52:44] "GET /authorize HTTP/1.1" 302 -
127.0.0.1 - - [03/Oct/2023 21:52:45] "GET /callback/?code=AQDhcAUwM66evIjaP94jGg1npMkWE7VWCB87tTA_fEOl_0hL4X4GY5czkfB84Ai9rpJf5-TMS2u2i304sJf8DBXFEWamuxhnPZCHvRGKeOLETIvZ4DfOJ4fqVwDJrRrXwKsmvd1540gGZQ1RPDePnFwNBvAjOzxWExnpyObTg76ID4g9wZfHfWKkHSl9KILGV0daCvmWB1ixWuBJbxLwXbvD1Any3YodzzZ9ZWC5MISdJSjwIPqGs_i8YGI&state=736ac0c806b49908cf44026f401b01 HTTP/1.1" 200 -
